## Validation
This Jupyter notebook includes all code for creating feature matrices and running a machine learning classifier. Some cells in this Jupyter notebook are nonfunctional, since some data files are too large for this github repository. Please adjust file paths as necessary.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from sklearn import linear_model
from sklearn.model_selection import train_test_split, cross_val_score, KFold, GroupKFold, GroupShuffleSplit, ShuffleSplit
from sklearn.metrics import f1_score, make_scorer, roc_curve
from sklearn.metrics import roc_auc_score, cohen_kappa_score
import sklearn.metrics as skm
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
import random
import csv
import preprocessing_utils
from preprocessing_utils import getPhenotypeData, getFeatureIntersectionDataSparse_PSP
from sklearn import feature_selection
import scipy
import pickle
from collections import Counter
%reload_ext autoreload
%autoreload 2

In [ ]:
featType1 = 'PSP_SimpleRepeats'
featType2 = 'SimpleRepeats'
X, index = getFeatureIntersectionDataSparse_PSP(featType1, featType2)
y = getPhenotypeData(X)

In [ ]:
print("Number Control: ", y.shape[0]-np.count_nonzero(y))
print("Number Cases: ", np.count_nonzero(y))

### Machine Learning Classifier
The following segment of code splits the data into train and test sets. A logistic regression classifier is trained on the training set and evaluated on the test set.

In [ ]:
with open('train_ind.pkl', 'rb') as f: #load from pre-saved training data
    train_ind = pickle.load(f)
with open('test_ind.pkl', 'rb') as f: #load from pre-saved training data
    test_ind = pickle.load(f)

X_train = X.getSampleSet(train_ind)
y_train = y[train_ind]
X_test = X.getSampleSet(test_ind)
y_test = y[test_ind]

print 'Size of Train X Matrix:', X_train.getMat().shape
print 'Size of Train y:', len(y_train)
print 'Size of Test X Matrix:', X_test.getMat().shape
print 'Size of Test y:', len(y_test)

Baseline Classifier - Guessing 0 for all or Guessing 1 for all

In [ ]:
total0 = 0
total1 = 0
for j in range(10):
    y_test = np.array(y_test)
    correct0 = 0
    correct1 = 0
    total = 0
    for i in range(len(y_test)):
        if(y_test[i]==0): correct0 += 1
        elif(y_test[i]==1): correct1 += 1
        total += 1.0
    total0 += (correct0/total)
    total1 += (correct1/total)
print 'Naive Approach - Guessing 0 for All: ', total0/10
print 'Naive Approach - Guessing 1 for All: ', total1/10

Logistic Regression Classifier - 5-Fold Cross Validation

In [ ]:
allPosVariants = []
allNegVariants = []
group_kfold = GroupKFold(n_splits=5)
trainsamples = list(X_train.getSamples())
train_famids = [str(x) for x in range(len(trainsamples))]

with open('data/v34.vcf.csv', 'rU') as csvfile:
    reader = csv.reader(csvfile, delimiter='\t')
    for row in reader:
        sample = row[0].split(',')
        if(sample[1] in trainsamples):
            train_famids[trainsamples.index(sample[1])] = sample[0]

#Evaluate across all five folds
k = 1
for train_kindex, val_index in group_kfold.split(X_train.getMat(), y_train, train_famids):
    X_ktrain = X_train.getSampleSet(train_kindex)
    y_ktrain = y_train[train_kindex]
    X_val = X_train.getSampleSet(val_index)
    y_val = y_train[val_index]   
    logreg = linear_model.LogisticRegression(penalty='l1', C=0.1, class_weight='balanced') #adjust for class imbalance
    print '\n5-Fold Cross Validation: Fold %d' % k
    logreg.fit(X_ktrain.getMat(), y_ktrain)
    print 'Train Accuracy:', logreg.score(X_ktrain.getMat(), y_ktrain)
    print 'Test Accuracy: ', logreg.score(X_val.getMat(), y_val)
    
    y_pred = logreg.predict(X_val.getMat())
    print 'Cohen Kappa Statistic: ', cohen_kappa_score(y_val, y_pred) #good metric for imbalanced classes
    print 'F1_Score: ', f1_score(y_val, y_pred)
    print 'Recall: ', skm.recall_score(y_val, y_pred)
    print 'Precision: ', skm.precision_score(y_val, y_pred)
    cnf_matrix = skm.confusion_matrix(y_val, y_pred)
    print 'Confusion Matrix: \n', cnf_matrix

    probas_ = logreg.fit(X_ktrain.getMat(), y_ktrain).decision_function(X_val.getMat())
    # Compute ROC curve and area the curve
    fpr, tpr, thresholds = roc_curve(y_val, probas_)
    plt.plot(fpr, tpr, color = 'darkorange', label = 'ROC curve')
    plt.plot([0,1], [0,1], color = 'navy', linestyle = '--')
    plt.show()
    
    plt.plot()
    precision, recall, _ = precision_recall_curve(abs(y_val-1), 1-probas_)
    plt.step(recall, precision, color='b', alpha=1, where='post')

    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.ylim([0.0, 1.05])
    plt.xlim([0.0, 1.0])
    average_precision = average_precision_score(y_val, probas_)
    plt.title('2-class Precision-Recall curve with Flipped Class Labels: Avg P={0:0.2f}'.format(average_precision))
    plt.show()


    negargs = list((logreg.coef_).argsort()[0, 0:5])
    posargs = list((logreg.coef_).argsort()[0, -5:])
    print 'Test AUC-ROC:', roc_auc_score(y_val, probas_)  

    topvalnegvariants = []
    topvalposvariants = []

    for elem in negargs:
        topvalnegvariants.append(X_ktrain.getFeatures()[elem])
        print logreg.coef_[0,elem], 
    for elem in posargs:
        topvalposvariants.append(X_ktrain.getFeatures()[elem])
        print logreg.coef_[0,elem], 

    print '\nTop Positive Variants: ',
    for elem in topvalposvariants:
        allPosVariants.append(elem)
        print elem,
    print '\nTop Negative Variants: ',
    for elem in topvalnegvariants:
        allNegVariants.append(elem)
        print elem,
    print ''

    k+=1

print '\nTop Positive variants that occur in exactly two folds: ', 
cnt = Counter(allPosVariants)
for elem in [k for k, v in cnt.iteritems() if v == 2]: print elem,
print '\nTop Positive variants that occur in exactly three folds: ', 
for elem in [k for k, v in cnt.iteritems() if v == 3]: print elem,
print '\nTop Positive variants that occur in exactly four folds: ', 
for elem in [k for k, v in cnt.iteritems() if v == 4]: print elem,
print '\nTop Positive variants that occur in exactly five folds: ', 
for elem in [k for k, v in cnt.iteritems() if v == 5]: print elem,

print '\nTop Negative variants that occur in exactly two folds: ', 
cnt = Counter(allNegVariants)
for elem in [k for k, v in cnt.iteritems() if v == 2]: print elem,
print '\nTop Negative variants that occur in exactly three folds: ', 
for elem in [k for k, v in cnt.iteritems() if v == 3]: print elem,
print '\nTop Negative variants that occur in exactly four folds: ', 
for elem in [k for k, v in cnt.iteritems() if v == 4]: print elem,
print '\nTop Negative variants that occur in exactly five folds: ', 
for elem in [k for k, v in cnt.iteritems() if v == 5]: print elem,


Logistic Regression Classifier - Entire training set and held-out test set

In [ ]:
logreg = linear_model.LogisticRegression(penalty='l1', C=0.1, class_weight='balanced') #adjust for class imbalance
print 'Training across entire set and using test set'
logreg.fit(X_train.getMat(), y_train)

print 'Train Accuracy: {0:0.3f}'.format(logreg.score(X_train.getMat(), y_train))
print 'Test Accuracy: {0:0.3f}'.format(logreg.score(X_test.getMat(), y_test))

y_pred = logreg.predict(X_test.getMat())
print 'Cohen Kappa Statistic: {0:0.3f}'.format(cohen_kappa_score(y_test, y_pred)) #good metric for imbalanced classes
print 'F1_Score: {0:0.3f}'.format(f1_score(y_test, y_pred))
print 'Recall: {0:0.3f}'.format(skm.recall_score(y_test, y_pred))
print 'Precision: {0:0.3f}'.format(skm.precision_score(y_test, y_pred))
cnf_matrix = skm.confusion_matrix(y_test, y_pred)
print 'Confusion Matrix: \n', cnf_matrix

probas_ = logreg.fit(X_train.getMat(), y_train).decision_function(X_test.getMat())
# Compute ROC curve and area the curve
fpr, tpr, thresholds = roc_curve(y_test, probas_)
plt.plot(fpr, tpr, color = 'darkorange', label = 'ROC curve')
plt.plot([0,1], [0,1], color = 'navy', linestyle = '--')
plt.show()

plt.plot()
precision, recall, _ = precision_recall_curve(abs(y_test-1), 1-probas_)
plt.step(recall, precision, color='b', alpha=1, where='post')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
average_precision = average_precision_score(y_test, probas_)
plt.title('2-class Precision-Recall curve with Flipped Class Labels: Avg P={0:0.2f}'.format(average_precision))
plt.show()


args = list((logreg.coef_).argsort()[0, 0:5])
args.extend((logreg.coef_).argsort()[0, -5:])
print np.count_nonzero(logreg.coef_)
print 'Test AUC-ROC: {0:0.3f}'.format(roc_auc_score(y_test, probas_))

topvariants = []
for elem in args:
    topvariants.append(X_train.getFeatures()[elem])
    print logreg.coef_[0,elem], 

print ''    
for elem in topvariants:
    print elem, 
print ''